# L3 M3.4: Incident Response & Breach Notification

## Learning Arc

**Purpose:** Build a production-grade incident response and breach notification system for GCC compliance environments, automating the detection, classification, response workflow, and regulatory notification process for security and compliance incidents.

**Concepts Covered:**
- 4-tier incident severity classification (P0 Critical, P1 High, P2 Medium, P3 Low)
- 6-phase incident response workflow (Detection, Containment, Investigation, Eradication, Recovery, Post-Mortem)
- Automated breach notification for GDPR Article 33/34 and DPDPA compliance
- Multi-tenant incident isolation and tracking
- Regulatory deadline management (72-hour GDPR notification requirement)
- Incident audit trail and documentation
- Root cause analysis and preventive measures

**After Completing This Notebook:**
- You will understand how to classify incident severity based on impact assessment
- You can implement a structured 6-phase incident response workflow
- You will recognize when regulatory breach notification is required
- You can automate GDPR/DPDPA notification with deadline tracking
- You will implement multi-tenant incident isolation
- You can generate post-mortem reports with lessons learned
- You will build production-ready incident response systems for GCC environments

**Context in Track L3.M3:**
This module builds on M3.3 (Audit Logging & SIEM) and completes the monitoring & reporting track by adding incident response capabilities. You've learned to log compliance events - now you'll handle security incidents and breach notifications.

In [ ]:
# Environment Setup
import os
import sys

# Add src to path for imports
if '../src' not in sys.path:
    sys.path.insert(0, '../src')

# This module has no external service dependencies
# All incident response processing is local
print("✓ No external services required")
print("  → All incident classification and workflow management is local")
print("  → Ready to explore incident response automation")

## Section 1: Understanding Incident Severity Classification

### Why Classification Matters

In GCC environments, not all incidents are equal. A configuration drift in a test environment (P3) requires different handling than a production data breach affecting 10,000 users (P0).

**The 4-Tier System:**
- **P0 (Critical):** Data breaches, full outages, regulatory violations - immediate escalation required
- **P1 (High):** Partial data exposure, service degradation, security incidents - urgent response
- **P2 (Medium):** Minor security events, isolated tenant issues - standard response
- **P3 (Low):** Low-impact incidents, configuration issues - routine handling

**Real-world Impact:**
- P0 incident: CFO gets paged at 2 AM, war room activated, regulatory lawyers on standby
- P3 incident: Ticket created, assigned to engineer, fixed during business hours

In [ ]:
from l3_m3_monitoring_reporting import IncidentClassifier, IncidentType, IncidentSeverity

# Example 1: P0 Classification - Large data breach
severity_p0 = IncidentClassifier.classify_incident(
    incident_type=IncidentType.DATA_BREACH,
    affected_users_count=2500,
    data_sensitivity="RESTRICTED",
    service_impact="PARTIAL"
)

print(f"Large data breach (2500 users, RESTRICTED data): {severity_p0}")
# Expected: P0_CRITICAL

# Example 2: P1 Classification - Unauthorized access
severity_p1 = IncidentClassifier.classify_incident(
    incident_type=IncidentType.UNAUTHORIZED_ACCESS,
    affected_users_count=150,
    data_sensitivity="CONFIDENTIAL",
    service_impact="PARTIAL"
)

print(f"Unauthorized access (150 users): {severity_p1}")
# Expected: P1_HIGH

# Example 3: P3 Classification - Low impact
severity_p3 = IncidentClassifier.classify_incident(
    incident_type=IncidentType.COMPLIANCE_FAILURE,
    affected_users_count=3,
    data_sensitivity="PUBLIC",
    service_impact="NONE"
)

print(f"Minor compliance issue (3 users, PUBLIC data): {severity_p3}")
# Expected: P3_LOW

## Section 2: Breach Notification Requirements

### When Notification is Required

**GDPR (EU):**
- Article 33: Notify Data Protection Authority within **72 hours** of becoming aware of breach
- Article 34: Notify affected users if **high risk** to rights and freedoms
- Penalties: Up to €20M or 4% of global annual revenue

**DPDPA (India):**
- Notify Data Protection Board within **reasonable time** (typically 72 hours)
- Notify affected individuals
- Penalties: Up to ₹250 crore

**Key Decision Point:**
Not all incidents require notification. Our classifier automatically determines this based on:
1. Severity (all P0 incidents require notification)
2. Incident type (DATA_BREACH and PII_EXPOSURE always require notification)
3. Data sensitivity and user impact

In [ ]:
# Check if notification is required

# P0 incident - always requires notification
notification_p0 = IncidentClassifier.requires_notification(
    severity=IncidentSeverity.P0,
    incident_type=IncidentType.SERVICE_OUTAGE
)
print(f"P0 service outage requires notification: {notification_p0}")
# Expected: True

# Data breach - always requires notification regardless of severity
notification_breach = IncidentClassifier.requires_notification(
    severity=IncidentSeverity.P2,
    incident_type=IncidentType.DATA_BREACH
)
print(f"P2 data breach requires notification: {notification_breach}")
# Expected: True

# P3 non-breach - no notification required
notification_p3 = IncidentClassifier.requires_notification(
    severity=IncidentSeverity.P3,
    incident_type=IncidentType.COMPLIANCE_FAILURE
)
print(f"P3 compliance failure requires notification: {notification_p3}")
# Expected: False

## Section 3: Phase 1 - Incident Detection

### Detection Sources
- **Automated monitoring:** SIEM alerts, anomaly detection, compliance scans
- **User reports:** Security team escalations, customer complaints
- **Audit findings:** Periodic security audits, penetration tests

### What Happens During Detection
1. Incident logged with unique ID
2. Severity automatically classified
3. Notification requirement determined
4. If notification required, 72-hour deadline set
5. Incident enters "ACTIVE" status

In [ ]:
from l3_m3_monitoring_reporting import IncidentResponseWorkflow
from datetime import datetime, timezone

# Create workflow instance
workflow = IncidentResponseWorkflow()

# Detect a data breach incident
incident = workflow.detect_incident(
    tenant_id="tenant-healthcare-demo",
    incident_type=IncidentType.DATA_BREACH,
    description="Unauthorized API access detected - customer PII exposed",
    detected_by="security-monitor",
    affected_users=[f"user-{i}" for i in range(1, 201)],  # 200 users
    affected_data_types=["email", "phone_number", "address"],
    data_sensitivity="CONFIDENTIAL",
    service_impact="PARTIAL"
)

print(f"Incident ID: {incident.incident_id}")
print(f"Severity: {incident.severity}")
print(f"Notification required: {incident.notification_required}")
print(f"Notification deadline: {incident.notification_deadline}")
print(f"Current phase: {incident.current_phase}")
print(f"Status: {incident.status}")

# Expected:
# Incident ID: INC-tena-<hash>
# Severity: P1_HIGH (200 users + CONFIDENTIAL data)
# Notification required: True (data breach)
# Notification deadline: ~72 hours from now
# Current phase: detection
# Status: ACTIVE

## Section 4: Phase 2 - Containment

### Goal: Stop the Bleeding

Containment prevents further damage while you investigate. Speed is critical - P0 incidents should be contained within 15 minutes.

**Common Containment Actions:**
- Disable compromised user accounts
- Revoke API keys and access tokens
- Block malicious IP addresses
- Isolate affected tenant/database (multi-tenant)
- Enable read-only mode on affected systems
- Activate Web Application Firewall (WAF) rules

**Trade-off:** Containment may cause service disruption for legitimate users. Balance speed with impact.

In [ ]:
# Contain the incident
containment_result = workflow.contain_incident(
    incident_id=incident.incident_id,
    containment_actions=[
        "Blocked IP address 203.0.113.45 (source of unauthorized access)",
        "Revoked API key: ak_live_abc123def456",
        "Disabled affected user accounts (200 users)",
        "Enabled database read-only mode for tenant-healthcare-demo",
        "Activated WAF rule: block-unauthorized-api-patterns"
    ]
)

print(f"Phase: {containment_result['phase']}")
print(f"Status: {containment_result['status']}")
print(f"Actions taken: {len(containment_result['actions_taken'])} actions")
print(f"Timestamp: {containment_result['timestamp']}")

# Verify phase updated
updated_incident = workflow.get_incident(incident.incident_id)
print(f"\nIncident now in phase: {updated_incident.current_phase}")

# Expected:
# Phase: containment
# Status: contained
# Actions taken: 5 actions
# Incident now in phase: containment

## Section 5: Phase 3 - Investigation

### Root Cause Analysis

Investigation determines:
1. **How did this happen?** (Attack vector, vulnerability exploited)
2. **What was affected?** (Full scope of data/systems compromised)
3. **Who was involved?** (Internal/external actors)
4. **When did it start?** (Timeline of events)

**Investigation Tasks:**
- Analyze audit logs (from M3.3 - Audit Logging module)
- Review SIEM correlation rules
- Interview affected users and staff
- Forensic analysis of affected systems
- Timeline reconstruction

**Output:** Detailed findings document for post-mortem

In [ ]:
# Conduct investigation
investigation_result = workflow.investigate_incident(
    incident_id=incident.incident_id,
    investigation_findings="""
    ROOT CAUSE: API authentication bypass vulnerability (CVE-2024-XXXXX)
    
    TIMELINE:
    - 2025-11-16 08:00 UTC: Attacker discovered authentication bypass in /api/v2/users endpoint
    - 2025-11-16 08:15 UTC: First unauthorized API call (GET /api/v2/users?limit=1000)
    - 2025-11-16 08:15-09:45 UTC: 47 API calls extracting 200 user records
    - 2025-11-16 09:50 UTC: SIEM alert triggered (unusual API pattern)
    - 2025-11-16 10:00 UTC: Security team investigation started
    
    SCOPE:
    - 200 user records accessed (email, phone, address)
    - No payment data accessed (stored in separate database)
    - No evidence of data exfiltration beyond API responses
    
    ATTACK VECTOR:
    - Publicly disclosed vulnerability in authentication library v2.3.1
    - Patch available (v2.3.2) but not yet deployed
    - IP address 203.0.113.45 (Amsterdam, VPN)
    """
)

print(f"Phase: {investigation_result['phase']}")
print(f"Findings summary:")
print(investigation_result['findings'][:200] + "...")  # First 200 chars

# Expected:
# Phase: investigation
# Findings: ROOT CAUSE: API authentication bypass vulnerability...

## Section 6: Phase 4 - Eradication

### Removing the Threat

Eradication eliminates the root cause so the incident can't recur.

**Common Eradication Actions:**
- **Patch vulnerabilities:** Apply security updates, upgrade libraries
- **Remove malware:** Clean infected systems, rebuild compromised servers
- **Reset credentials:** Force password resets, rotate all secrets
- **Update policies:** Fix RBAC misconfigurations, tighten access controls
- **Harden systems:** Enable additional security features (2FA, WAF, rate limiting)

**Validation:** Test that the vulnerability is actually fixed before recovery.

In [ ]:
# Eradicate the threat
eradication_result = workflow.eradicate_threat(
    incident_id=incident.incident_id,
    eradication_actions=[
        "Upgraded authentication library from v2.3.1 to v2.3.2 (patched vulnerability)",
        "Deployed patch to production (3 API servers)",
        "Rotated all API keys (1,234 keys invalidated, new keys issued)",
        "Implemented rate limiting: 100 requests/minute per API key",
        "Added input validation to /api/v2/users endpoint",
        "Enabled API request logging for all /api/v2/* endpoints",
        "Verified vulnerability no longer exploitable via penetration test"
    ]
)

print(f"Phase: {eradication_result['phase']}")
print(f"Actions taken: {len(eradication_result['actions_taken'])} eradication actions")
print("\nKey actions:")
for i, action in enumerate(eradication_result['actions_taken'][:3], 1):
    print(f"  {i}. {action[:80]}...")  # First 80 chars

# Expected:
# Phase: eradication
# Actions taken: 7 eradication actions
# Key actions:
#   1. Upgraded authentication library from v2.3.1 to v2.3.2...
#   2. Deployed patch to production (3 API servers)...
#   3. Rotated all API keys (1,234 keys invalidated, new keys issued)...

## Section 7: Phase 5 - Recovery

### Restoring Normal Operations

Recovery brings systems back online after eradication is complete.

**Recovery Steps:**
1. **Re-enable services:** Disable read-only mode, restore write access
2. **Restore user access:** Re-enable accounts (with new credentials)
3. **Verify integrity:** Ensure no lingering compromise
4. **Monitor closely:** Watch for signs of recurrence (24-48 hours)
5. **Communicate:** Notify users that service is restored

**Validation Checklist:**
- [ ] All systems operational
- [ ] All security patches applied
- [ ] Monitoring in place for recurrence
- [ ] Users notified of service restoration
- [ ] SLA compliance verified

In [ ]:
# Recover services
recovery_result = workflow.recover_services(
    incident_id=incident.incident_id,
    recovery_steps=[
        "Disabled read-only mode for tenant-healthcare-demo database",
        "Re-enabled API access with new rate limits",
        "Re-enabled 200 affected user accounts (new API keys issued)",
        "Sent email to affected users: 'Service restored, please use new API key'",
        "Verified all API endpoints responding normally (200 OK)",
        "Enabled enhanced monitoring: alert if >50 API calls/min from single IP",
        "Scheduled 24-hour incident watch (security team on-call)"
    ]
)

print(f"Phase: {recovery_result['phase']}")
print(f"Recovery steps completed: {len(recovery_result['recovery_steps'])}")
print(f"Timestamp: {recovery_result['timestamp']}")

# Verify incident is in recovery phase
updated_incident = workflow.get_incident(incident.incident_id)
print(f"\nIncident status: {updated_incident.status}")
print(f"Current phase: {updated_incident.current_phase}")

# Expected:
# Phase: recovery
# Recovery steps completed: 7
# Incident status: ACTIVE (still open, awaiting post-mortem)
# Current phase: recovery

## Section 8: Phase 6 - Post-Mortem and Closure

### Learning from Incidents

Post-mortem documents what happened and ensures it doesn't happen again.

**Post-Mortem Contents:**
1. **Timeline:** Complete sequence of events from detection to resolution
2. **Root cause:** Why the incident occurred (technical + process failures)
3. **Lessons learned:** What went well, what didn't
4. **Preventive measures:** Specific actions to prevent recurrence
5. **Process improvements:** Updates to incident response runbooks

**Key Principle: Blameless Post-Mortems**
- Focus on systems, not individuals
- Goal: Improve processes, not punish people
- Create psychological safety for honest reporting

**After Post-Mortem:**
- Incident status changes from "ACTIVE" to "CLOSED"
- Resolution time recorded
- Preventive measures tracked to completion

In [ ]:
# Close incident with post-mortem
postmortem_result = workflow.close_with_post_mortem(
    incident_id=incident.incident_id,
    lessons_learned="""
    WHAT WENT WELL:
    - SIEM detected unusual API pattern within 1 hour 35 minutes
    - Containment completed within 15 minutes of detection
    - Multi-tenant isolation prevented other tenants from being affected
    - Security patch available and deployed within 4 hours
    
    WHAT DIDN'T GO WELL:
    - Authentication library vulnerability was publicly disclosed 2 weeks ago
    - Patch (v2.3.2) was available but not deployed (missed in sprint planning)
    - No automated dependency scanning in CI/CD pipeline
    - API rate limiting was not enabled (could have slowed attacker)
    
    ROOT CAUSE ANALYSIS:
    Technical: CVE-2024-XXXXX in auth library v2.3.1 (known vulnerability)
    Process: No automated patch management for critical dependencies
    
    IMPACT:
    - 200 users affected (PII exposed)
    - 1 hour 45 minutes of service disruption (read-only mode)
    - Estimated cost: $15K (incident response time + user notifications)
    - GDPR notification required (breach reported to DPA)
    """,
    preventive_measures=[
        "Implement automated dependency scanning (Dependabot, Snyk) in CI/CD",
        "Create weekly security patch review process (every Monday)",
        "Enable API rate limiting by default: 100 req/min per key",
        "Add SIEM alert: notify if critical CVE affects our stack within 24 hours",
        "Conduct quarterly incident response drills (tabletop exercises)",
        "Update runbook: authentication vulnerabilities are P0 for patching",
        "Schedule penetration test: focus on API authentication bypass scenarios"
    ]
)

print(f"Phase: {postmortem_result['phase']}")
print(f"Detected at: {postmortem_result['detected_at']}")
print(f"Resolved at: {postmortem_result['resolved_at']}")
print(f"Total duration: {postmortem_result['duration']}")
print(f"\nPreventive measures: {len(postmortem_result['preventive_measures'])} actions")

# Verify incident is closed
final_incident = workflow.get_incident(incident.incident_id)
print(f"\nFinal status: {final_incident.status}")
print(f"Final phase: {final_incident.current_phase}")

# Expected:
# Phase: post_mortem
# Total duration: ~few seconds (in demo)
# Preventive measures: 7 actions
# Final status: CLOSED
# Final phase: post_mortem

## Section 9: Breach Notification Automation

### GDPR Article 33 - Regulatory Notification

When a data breach requires notification (P0/P1 data breaches), you must notify the Data Protection Authority within 72 hours.

**Required Information:**
1. Nature of the breach (what happened)
2. Categories of data subjects affected (users, customers, employees)
3. Approximate number of affected individuals
4. Data categories involved (email, phone, financial, health)
5. Likely consequences of the breach
6. Measures taken to address the breach
7. Contact point for more information

**Automation Benefits:**
- Ensures notification sent within deadline
- Standardized notification format
- Automatic inclusion of all required fields
- Proof of timely notification (audit trail)

In [ ]:
# Send GDPR breach notification to DPA
gdpr_notification = workflow.send_breach_notification(
    incident_id=incident.incident_id,
    recipient="dpa@example.eu",
    notification_type="REGULATORY",
    regulation="GDPR"
)

print(f"Notification ID: {gdpr_notification.notification_id}")
print(f"Sent to: {gdpr_notification.recipient}")
print(f"Notification type: {gdpr_notification.notification_type}")
print(f"Regulation: {gdpr_notification.regulation}")
print(f"Sent at: {gdpr_notification.sent_at}")
print(f"\nNotification content (preview):")
print(gdpr_notification.notification_content[:300] + "...")  # First 300 chars

# Expected:
# Notification ID: NOT-<hash>
# Sent to: dpa@example.eu
# Notification type: REGULATORY
# Regulation: GDPR
# Content includes: GDPR Article 33, incident details, affected users, data types

### DPDPA - India Breach Notification

Similar to GDPR, India's Digital Personal Data Protection Act (DPDPA) requires notification to the Data Protection Board of India.

In [ ]:
# Send DPDPA breach notification
dpdpa_notification = workflow.send_breach_notification(
    incident_id=incident.incident_id,
    recipient="dpb@india.gov",
    notification_type="REGULATORY",
    regulation="DPDPA"
)

print(f"DPDPA Notification sent to: {dpdpa_notification.recipient}")
print(f"Regulation: {dpdpa_notification.regulation}")
print(f"\nNotification content (preview):")
print(dpdpa_notification.notification_content[:250] + "...")  # First 250 chars

# Expected:
# Notification includes: DPDPA reference, incident details, affected individuals

### User Notification (GDPR Article 34)

If the breach poses a **high risk** to individuals' rights and freedoms, you must also notify affected users directly.

In [ ]:
# Send user notification
user_notification = workflow.send_breach_notification(
    incident_id=incident.incident_id,
    recipient="affected.users@example.com",
    notification_type="USER",
    regulation="GDPR"
)

print(f"User notification sent to: {user_notification.recipient}")
print(f"Notification type: {user_notification.notification_type}")

# List all notifications for this incident
all_notifications = workflow.notifications
incident_notifications = [n for n in all_notifications if n.incident_id == incident.incident_id]

print(f"\nTotal notifications sent for {incident.incident_id}: {len(incident_notifications)}")
for notif in incident_notifications:
    print(f"  - {notif.notification_type}: {notif.recipient} ({notif.regulation})")

# Expected:
# Total notifications: 3 (DPA GDPR, DPA DPDPA, User)
#   - REGULATORY: dpa@example.eu (GDPR)
#   - REGULATORY: dpb@india.gov (DPDPA)
#   - USER: affected.users@example.com (GDPR)

## Section 10: Multi-Tenant Incident Isolation

### Why Multi-Tenant Isolation Matters

In GCC environments, you manage incidents for multiple customers (tenants). Critical requirements:
1. **Data isolation:** Tenant A can't see Tenant B's incidents
2. **Independent workflows:** Each tenant's incidents are tracked separately
3. **Separate notifications:** Breach in Tenant A doesn't trigger notification for Tenant B

### Implementation
Every incident is tagged with `tenant_id`. All queries filter by tenant.

In [ ]:
# Create incidents for different tenants

# Tenant A: Healthcare company (P0 incident)
incident_a = workflow.detect_incident(
    tenant_id="tenant-healthcare-a",
    incident_type=IncidentType.DATA_BREACH,
    description="HIPAA violation - patient records exposed",
    detected_by="compliance-scan",
    affected_users=[f"patient-{i}" for i in range(1000)],
    affected_data_types=["health_records", "ssn"],
    data_sensitivity="RESTRICTED",
    service_impact="PARTIAL"
)

# Tenant B: E-commerce company (P3 incident)
incident_b = workflow.detect_incident(
    tenant_id="tenant-ecommerce-b",
    incident_type=IncidentType.COMPLIANCE_FAILURE,
    description="Cookie consent banner missing on checkout page",
    detected_by="automated-test",
    affected_users=[f"customer-{i}" for i in range(5)],
    affected_data_types=["cookie_preferences"],
    data_sensitivity="PUBLIC",
    service_impact="NONE"
)

# Tenant C: Financial services (P1 incident)
incident_c = workflow.detect_incident(
    tenant_id="tenant-fintech-c",
    incident_type=IncidentType.UNAUTHORIZED_ACCESS,
    description="Failed brute force attack on admin panel",
    detected_by="security-monitor",
    affected_users=[f"admin-{i}" for i in range(3)],
    affected_data_types=["login_credentials"],
    data_sensitivity="CONFIDENTIAL",
    service_impact="NONE"
)

print(f"Tenant A incident: {incident_a.incident_id} - {incident_a.severity}")
print(f"Tenant B incident: {incident_b.incident_id} - {incident_b.severity}")
print(f"Tenant C incident: {incident_c.incident_id} - {incident_c.severity}")

# Expected:
# Tenant A: P0_CRITICAL (1000 users, RESTRICTED health data)
# Tenant B: P3_LOW (5 users, PUBLIC data)
# Tenant C: P1_HIGH (unauthorized access, CONFIDENTIAL)

In [ ]:
# Query incidents by tenant - demonstrate isolation

# Tenant A incidents only
tenant_a_incidents = workflow.list_incidents(tenant_id="tenant-healthcare-a")
print(f"Tenant A incidents: {len(tenant_a_incidents)}")
for inc in tenant_a_incidents:
    print(f"  - {inc.incident_id}: {inc.description[:50]}...")

# Tenant B incidents only
tenant_b_incidents = workflow.list_incidents(tenant_id="tenant-ecommerce-b")
print(f"\nTenant B incidents: {len(tenant_b_incidents)}")
for inc in tenant_b_incidents:
    print(f"  - {inc.incident_id}: {inc.description[:50]}...")

# All P0 incidents across all tenants (admin view)
p0_incidents = workflow.list_incidents(severity=IncidentSeverity.P0)
print(f"\nAll P0 incidents (admin view): {len(p0_incidents)}")
for inc in p0_incidents:
    print(f"  - {inc.incident_id} ({inc.tenant_id}): {inc.description[:40]}...")

# Expected:
# Tenant A: 1 incident (healthcare breach)
# Tenant B: 1 incident (cookie compliance)
# P0 incidents: 1-2 incidents (depends on earlier examples)

## Section 11: Production Best Practices

### Incident Response Readiness

**Before Production:**
1. **Test the workflow:** Run incident response drills (tabletop exercises)
2. **Train the team:** Everyone knows their role in P0 incidents
3. **Set up integrations:** SIEM alerts → automated incident detection
4. **Define escalation paths:** P0 → page CEO, P1 → page security team, etc.
5. **Prepare notification templates:** Pre-approved by legal team

**During Incident:**
1. **Stay calm:** Follow the 6-phase workflow
2. **Document everything:** Every action logged with timestamp
3. **Communicate proactively:** Update stakeholders every hour for P0
4. **Don't skip phases:** Even if under pressure

**After Incident:**
1. **Complete post-mortem:** Within 48 hours of resolution
2. **Track preventive measures:** Assign owners, set deadlines
3. **Update runbooks:** Incorporate lessons learned
4. **Share knowledge:** Present post-mortem to engineering team

In [ ]:
# Production readiness check

# Count active incidents by severity
active_incidents = workflow.list_incidents(status="ACTIVE")
p0_active = [i for i in active_incidents if i.severity == IncidentSeverity.P0]
p1_active = [i for i in active_incidents if i.severity == IncidentSeverity.P1]

print("=" * 50)
print("INCIDENT RESPONSE DASHBOARD")
print("=" * 50)
print(f"Total incidents tracked: {len(workflow.incidents)}")
print(f"Active incidents: {len(active_incidents)}")
print(f"  - P0 (Critical): {len(p0_active)}")
print(f"  - P1 (High): {len(p1_active)}")
print(f"\nClosed incidents: {len(workflow.list_incidents(status='CLOSED'))}")
print(f"Total notifications sent: {len(workflow.notifications)}")

# Check for pending notifications (deadline approaching)
pending_notifications = [
    i for i in workflow.incidents.values()
    if i.notification_required and i.notification_deadline and i.status == "ACTIVE"
]

if pending_notifications:
    print(f"\n⚠️ WARNING: {len(pending_notifications)} incidents require notification:")
    for inc in pending_notifications:
        print(f"  - {inc.incident_id}: Deadline {inc.notification_deadline}")
else:
    print("\n✓ No pending notifications")

print("=" * 50)

# Expected output will vary based on incidents created in this session

## Section 12: Compliance Checklist

### Regulatory Requirements Covered

**GDPR (EU):**
- ✓ Article 33: DPA notification within 72 hours
- ✓ Article 34: User notification if high risk
- ✓ Article 32: Security incident response procedures
- ✓ Article 5: Accountability (documented incident handling)

**DPDPA (India):**
- ✓ Section 6: Data breach notification to Data Protection Board
- ✓ Section 8: Notification to data principals (users)
- ✓ Reasonable security practices (incident response workflow)

**SOC 2 (Trust Services Criteria):**
- ✓ CC7.3: Security incident detection and response
- ✓ CC7.4: Incident response plan
- ✓ CC9.1: Risk mitigation activities

**ISO 27001:**
- ✓ A.16: Information security incident management
- ✓ A.16.1.4: Assessment and decision on information security events
- ✓ A.16.1.5: Response to information security incidents
- ✓ A.16.1.7: Collection of evidence

## Summary

You've built a production-ready incident response system that:

1. **Classifies incidents** automatically (P0-P3) based on impact
2. **Enforces 6-phase workflow** from detection to post-mortem
3. **Automates breach notifications** for GDPR and DPDPA compliance
4. **Isolates multi-tenant incidents** for GCC environments
5. **Tracks regulatory deadlines** (72-hour GDPR notification)
6. **Documents everything** for audit trail and compliance

### Next Steps

1. **Integrate with SIEM:** Connect your monitoring (M3.3) to automated incident detection
2. **Customize thresholds:** Adjust P0/P1/P2/P3 criteria for your risk tolerance
3. **Add notification integrations:** Email/SMS for user notifications, PagerDuty for P0 alerts
4. **Run drills:** Practice incident response with tabletop exercises
5. **Deploy to production:** Start with non-critical tenants, gradually expand

### Key Takeaways

- **Speed matters:** P0 containment in <15 minutes prevents damage
- **Don't skip phases:** Investigation before eradication (understand root cause first)
- **Documentation is insurance:** Post-mortems protect you in audits
- **Automation reduces errors:** Manual notification = missed deadlines
- **Multi-tenant isolation is critical:** Never leak one tenant's incident to another

You're now ready to handle security incidents and breach notifications in production GCC RAG systems!